<a href="https://colab.research.google.com/github/plushvoxel/Project-Lernende-Agenten-colab/blob/master/2018_11_18_marble_3classes_512samples_crossFeatures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from __future__ import print_function

import math
from urllib import request
from IPython import display
from matplotlib import cm
from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import metrics
import tensorflow as tf
from tensorflow.python.data import Dataset
from google.colab import files
from tarfile import open as taropen
from struct import unpack
import os
import glob
import math
import seaborn as sns
import time
import shutil
!pip install XlsxWriter


In [4]:
tf.logging.set_verbosity(tf.logging.ERROR)
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.1f}'.format


data = dict()
MODKEY = "mod"
Classes = ["fm", "pager", "smartwares"]
totalDivAverageLabel = "totalDivAverage"
ClassesAmount = len(Classes)
FeaturesAmount = 1025
for modulation in range(ClassesAmount):
  filename = "{}.tar".format(Classes[modulation])
  request.urlretrieve("https://marble.adhara.uberspace.de/{}_512.tar".format(Classes[modulation]), filename)
  tar = taropen(filename)
  
  for member in tar.getmembers():
    if not MODKEY in data:
      data[MODKEY] = [modulation]
    else:
      data[MODKEY].append(modulation)
    with tar.extractfile(member) as f:
      maxAmplitude = 0
      totalAmplitude = 0
      totalSamples = 0
      buffer = f.read()
      num_floats = len(buffer)//4
      floats = unpack("f"*num_floats, buffer)
      # seperate imaginary and quadrature parts
      i = floats[0::2]
      q = floats[1::2]
      for j in range(min(len(i), len(q))):
        totalSamples = totalSamples +1
        ikey = "i{:05d}".format(j)
        qkey = "q{:05d}".format(j)
        if q[j] > maxAmplitude:
          maxAmplitude = q[j]
        totalAmplitude = totalAmplitude + q[j]
        if not ikey in data:
          data[ikey] = [i[j]]
        else:
          data[ikey].append(i[j])
        
        if not qkey in data:
          data[qkey] = [q[j]]
        else:
          data[qkey].append(q[j])    
      totalDivAverage = maxAmplitude - totalAmplitude/totalSamples
      if not totalDivAverageLabel in data:
        data[totalDivAverageLabel] = [totalDivAverage]
      else:
        data[totalDivAverageLabel].append(totalDivAverage)
signal_dataframe = pd.DataFrame(data=data)
signal_dataframeReal = signal_dataframe.copy()
signal_dataframe = signal_dataframe.reindex(np.random.permutation(signal_dataframe.index))
print(signal_dataframe)



       i00000  i00001  i00002  i00003  i00004  i00005  i00006  i00007  i00008  \
17896    -0.0     0.0     0.0    -0.0    -0.0    -0.0     0.0    -0.0     0.1   
15410     0.0    -0.0    -0.0     0.0    -0.0     0.0    -0.0     0.0    -0.1   
22636    -0.0    -0.0     0.0    -0.0    -0.1     0.1     0.0    -0.1     0.1   
27109     0.0    -0.0     0.0     0.0    -0.0     0.0    -0.0    -0.0     0.0   
14457    -0.0     0.0    -0.0     0.0    -0.0     0.0    -0.1     0.0     0.0   
...       ...     ...     ...     ...     ...     ...     ...     ...     ...   
21387     0.0    -0.0     0.0    -0.0    -0.1     0.2    -0.1     0.0     0.0   
12964    -0.0     0.0    -0.0     0.0    -0.0     0.0    -0.0     0.1    -0.0   
10509    -0.0     0.0    -0.0     0.0     0.0    -0.0    -0.0     0.0    -0.0   
5624     -0.0    -0.0     0.0    -0.0    -0.0     0.0    -0.0    -0.0     0.0   
24568     0.0    -0.0     0.0    -0.0     0.1    -0.2     0.1     0.1    -0.4   

       i00009       ...    

In [0]:
def parse_labels_and_features(dataset):
  """Extracts labels and features.
  
  This is a good place to scale or transform the features if needed.
  
  Args:
    dataset: A Pandas `Dataframe`, containing the label on the first column and
      monochrome pixel values on the remaining columns, in row major order.
  Returns:
    A `tuple` `(labels, features)`:
      labels: A Pandas `Series`.
      features: A Pandas `DataFrame`.
  """
  labels = dataset[MODKEY]

  # DataFrame.loc index ranges are inclusive at both ends.
  features = dataset.iloc[:,1:4097]
  return labels, features

In [0]:
def construct_feature_columns():
  """Construct the TensorFlow Feature Columns.

  Returns:
    A set of feature columns
  """ 
  
  # There are 784 pixels in each image.
  return set([tf.feature_column.numeric_column('features', shape=FeaturesAmount)])

In [0]:
def create_predict_input_fn(features, labels, batch_size, repeat_count = 1):
  """A custom input_fn for sending mnist data to the estimator for predictions.

  Args:
    features: The features to base predictions on.
    labels: The labels of the prediction examples.

  Returns:
    A function that returns features and labels for predictions.
  """
  def _input_fn():
    raw_features = {"features": features.values}
    raw_targets = np.array(labels)
    
    ds = Dataset.from_tensor_slices((raw_features, raw_targets)) # warning: 2GB limit
    ds = ds.batch(batch_size).repeat(repeat_count)
    
        
    # Return the next batch of data.
    feature_batch, label_batch = ds.make_one_shot_iterator().get_next()
    return feature_batch, label_batch

  return _input_fn

In [0]:
def create_training_input_fn(features, labels, batch_size, num_epochs=None, shuffle=True, repeat_count=1):
  """A custom input_fn for sending MNIST data to the estimator for training.

  Args:
    features: The training features.
    labels: The training labels.
    batch_size: Batch size to use during training.

  Returns:
    A function that returns batches of training features and labels during
    training.
  """
  def _input_fn(num_epochs=None, shuffle=True):
    # Input pipelines are reset with each call to .train(). To ensure model
    # gets a good sampling of data, even when number of steps is small, we 
    # shuffle all the data before creating the Dataset object
    idx = np.random.permutation(features.index)
    raw_features = {"features":features.reindex(idx)}
    raw_targets = np.array(labels[idx])
   
    ds = Dataset.from_tensor_slices((raw_features,raw_targets)) # warning: 2GB limit
    ds = ds.batch(batch_size).repeat(repeat_count)
    
    if shuffle:
      ds = ds.shuffle(10000)
    
    # Return the next batch of data.
    feature_batch, label_batch = ds.make_one_shot_iterator().get_next()
    return feature_batch, label_batch

  return _input_fn

In [0]:
def prepareWorksheet(worksheet):
  worksheet.write(0,0, 'Model')
  worksheet.write(0,1, 'learning rate')
  worksheet.write(0,2, 'training set size')
  worksheet.write(0,3, 'validating set size')
  worksheet.write(0,4, 'batchsize')
  worksheet.write(0,5, 'accuracy')
  worksheet.write(0,6, 'validation1')
  worksheet.write(0,7, 'validation2')
  worksheet.write(0,8, 'validation3')
  worksheet.write(0,9, 'validation4')
  worksheet.write(0,10, 'validation5')
  worksheet.write(0,11, 'validation6')
  worksheet.write(0,12, 'validation7')
  worksheet.write(0,13, 'validation8')
  worksheet.write(0,14, 'validation9')
  worksheet.write(0,15, 'validation10')
  worksheet.write(0,16, 'training1')
  worksheet.write(0,17, 'training2')
  worksheet.write(0,18, 'training3')
  worksheet.write(0,19, 'training4')
  worksheet.write(0,20, 'training5')
  worksheet.write(0,21, 'training6')
  worksheet.write(0,22, 'training7')
  worksheet.write(0,23, 'training8')
  worksheet.write(0,24, 'training9')
  worksheet.write(0,25, 'training10')
  

In [0]:
def train_nn_classification_model(
    learning_rate,
    steps,
    batch_size,
    hidden_units,
    training_examples,
    training_targets,
    validation_examples,
    validation_targets,
    test_examples,
    test_targets,
    worksheet,
    line,
    time_string):
  """Trains a neural network classification model for the MNIST digits dataset.
  
  In addition to training, this function also prints training progress information,
  a plot of the training and validation loss over time, as well as a confusion
  matrix.
  
  Args:
    learning_rate: An `int`, the learning rate to use.
    steps: A non-zero `int`, the total number of training steps. A training step
      consists of a forward and backward pass using a single batch.
    batch_size: A non-zero `int`, the batch size.
    hidden_units: A `list` of int values, specifying the number of neurons in each layer.
    training_examples: A `DataFrame` containing the training features.
    training_targets: A `DataFrame` containing the training labels.
    validation_examples: A `DataFrame` containing the validation features.
    validation_targets: A `DataFrame` containing the validation labels.
      
  Returns:
    The trained `DNNClassifier` object.
  """

  periods = 10
  # Caution: input pipelines are reset with each call to train. 
  # If the number of steps is small, your model may never see most of the data.  
  # So with multiple `.train` calls like this you may want to control the length 
  # of training with num_epochs passed to the input_fn. Or, you can do a really-big shuffle, 
  # or since it's in-memory data, shuffle all the data in the `input_fn`.
  steps_per_period = steps / periods  
  # Create the input functions.
  predict_training_input_fn = create_predict_input_fn(
    training_examples, training_targets, batch_size)
  predict_validation_input_fn = create_predict_input_fn(
    validation_examples, validation_targets, batch_size)
  predict_test_input_fn = create_predict_input_fn(
    test_examples, test_targets, batch_size)
  training_input_fn = create_training_input_fn(
    training_examples, training_targets, batch_size)
  
  
  # Create feature columns.
  feature_columns = [tf.feature_column.numeric_column('features', shape=FeaturesAmount)]

  # Create a DNNClassifier object.
  my_optimizer = tf.train.AdagradOptimizer(learning_rate=learning_rate)
  my_optimizer = tf.contrib.estimator.clip_gradients_by_norm(my_optimizer, 5.0)
  classifier = tf.estimator.DNNClassifier(
      feature_columns=feature_columns,
      n_classes=ClassesAmount,
      hidden_units=hidden_units,
      optimizer=my_optimizer,
      config=tf.contrib.learn.RunConfig(keep_checkpoint_max=10,
                                        model_dir="models")
  )

  # Train the model, but do so inside a loop so that we can periodically assess
  # loss metrics.
  print("Training model...")
  print("LogLoss error (on validation data):")
  training_errors = []
  validation_errors = []
  
  for period in range (0, periods):
    # Train the model, starting from the prior state.
    classifier.train(
        input_fn=training_input_fn,
        steps=steps_per_period
    )
    
    name= str(classifier.latest_checkpoint())
    oldfilemeta = name + ".meta"
    oldfiledata1 = name + ".data-00000-of-00002"
    oldfiledata2 = name + ".data-00001-of-00002"
    newfile = "saves/ckp"+str(period)+time_string
    shutil.copy(oldfilemeta, newfile+".meta")
    shutil.copy(oldfiledata1, newfile+".data1")
    shutil.copy(oldfiledata2, newfile+".data2")
    # Take a break and compute probabilities.
    training_predictions = list(classifier.predict(input_fn=predict_training_input_fn))
    training_probabilities = np.array([item['probabilities'] for item in training_predictions])
    training_pred_class_id = np.array([item['class_ids'][0] for item in training_predictions])
    training_pred_one_hot = tf.keras.utils.to_categorical(training_pred_class_id,ClassesAmount)
        
    validation_predictions = list(classifier.predict(input_fn=predict_validation_input_fn))
    validation_probabilities = np.array([item['probabilities'] for item in validation_predictions])    
    validation_pred_class_id = np.array([item['class_ids'][0] for item in validation_predictions])
    validation_pred_one_hot = tf.keras.utils.to_categorical(validation_pred_class_id,ClassesAmount)
    
    test_predictions = list(classifier.predict(input_fn=predict_test_input_fn))
    test_probabilities = np.array([item['probabilities'] for item in test_predictions])    
    test_pred_class_id = np.array([item['class_ids'][0] for item in test_predictions])
    test_pred_one_hot = tf.keras.utils.to_categorical(test_pred_class_id,ClassesAmount)   
    
    # Compute training and validation errors.
    training_log_loss = metrics.log_loss(training_targets, training_pred_one_hot)
    validation_log_loss = metrics.log_loss(validation_targets, validation_pred_one_hot)
    # Occasionally print the current loss.
    print("  period %02d : %0.2f" % (period, validation_log_loss))
    worksheet.write(line,(6+period), validation_log_loss)
    worksheet.write(line,(16+period),training_log_loss)
    # Add the loss metrics from this period to our list.
    training_errors.append(training_log_loss)
    validation_errors.append(validation_log_loss)
  model_general= []
  val_log_len = len(validation_errors)
  model_general.append(validation_errors[0]+ 2*abs(validation_errors[0]-validation_errors[1]))
  for index in range(1, val_log_len-1):
    model_general.append(validation_errors[index]+ abs(validation_errors[index]-validation_errors[index-1]) +  abs(validation_errors[index]-validation_errors[index+1]))
  model_general.append(validation_errors[val_log_len-1]+ 2*abs(validation_errors[val_log_len-1]-validation_errors[val_log_len-2]))
  index = np.argmin(model_general)
  savemodel_name = "ckp"+ str(index) + time_string
  shutil.copy("saves/"+savemodel_name+".meta", savemodel_name+".meta")
  shutil.copy("saves/"+savemodel_name+ ".data1", savemodel_name+".data1")
  shutil.copy("saves/"+savemodel_name+".data2", savemodel_name+".data2")
  shutil.rmtree('saves')
  os.makedirs("saves")
  print("Model training finished.")
  # Remove event files to save disk space.
  _ = map(os.remove, glob.glob(os.path.join(classifier.model_dir, 'events.out.tfevents*')))
  
  # Calculate final predictions (not probabilities, as above).
  final_predictions = classifier.predict(input_fn=predict_test_input_fn)
  final_predictions = np.array([item['class_ids'][0] for item in final_predictions])
  
  
  accuracy = metrics.accuracy_score(test_targets, final_predictions)
  print("Final accuracy (validation data): %0.2f" % accuracy)
  # Output a graph of loss metrics over periods.
  plt.ylabel("LogLoss")
  plt.xlabel("Periods")
  plt.title("LogLoss vs. Periods")
  plt.plot(training_errors, label="training")
  plt.plot(validation_errors, label="validation")
  plt.legend()
  pic1 = "{}.png".format(line)
  plt.savefig(pic1)
  plt.show()
  
  # Output a plot of the confusion matrix.
  cm = metrics.confusion_matrix(test_targets, final_predictions)
  # Normalize the confusion matrix by row (i.e by the number of samples
  # in each class).
  cm_normalized = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis]
  ax = sns.heatmap(cm_normalized, cmap="bone_r")
  ax.set_aspect(1)
  plt.title("Confusion matrix")
  plt.ylabel("True label")
  plt.xlabel("Predicted label")
  plt.show()
  #pic2 = "plt2.png"
  #plt.savefig(pic2)
  model = ""+str(hidden_units).strip('[]')+"]"
  insertIntoWorksheet(worksheet, line, model, learning_rate, training_examples.shape[0],validation_examples.shape[0], batch_size , accuracy)
  print(model)
  print(learning_rate)
  print(training_examples.shape[0])
  print(validation_examples.shape[0])
  print(batch_size)
  print(accuracy)
  plt.close(pic1)
  #plt.close(pic2)
  return classifier, accuracy

In [0]:
def insertIntoWorksheet(worksheet, line,hidden_units, learning_rate, training_examples_size,validating_examples_size, batch_size , accuracy):
  worksheet.write(line,0, hidden_units)
  worksheet.write(line,1, learning_rate)
  worksheet.write(line,2, training_examples_size)
  worksheet.write(line,3, validating_examples_size)
  worksheet.write(line,4, batch_size) 
  worksheet.write(line,5, accuracy)


In [0]:
def train_automated(
    training_set_size,
    validating_set_size,
    test_set_size,
    learning_rate,
    steps,
    batch_size,
    model,
    worksheet,
    line,
    time_str):
  """ Function used for automate the process of trying new network configurations
  
  Args:
    training_set_size: An 'int', number of samples used for training
    validating_set_size: An 'int', number of samples used for validation
    test_set_size: An 'int', number of samples used for test
    learning_rate: An `int`, the learning rate to use.
    steps: A non-zero `int`, the total number of training steps. A training step
      consists of a forward and backward pass using a single batch.
    batch_size: A non-zero `int`, the batch size.
    model: A list of 'int', define the number of neurons in each hidden layer
      
  Returns:
    The trained `DNNClassifier` object.
  """
  activation_function = "RELU" #@param ["RELU", "Sigmoid", "Tanh"]
  regression = "None" #@param ["None", "L1", "L2"]
  regression_rate = 3 #@param ["3", "1", "0.3", "0.1", "0.03", "0.01", "0.003", "0.001"] {type:"raw"}
  training_targets, training_examples = parse_labels_and_features(signal_dataframe[0:training_set_size])
  validation_targets, validation_examples = parse_labels_and_features(signal_dataframe[training_set_size:(training_set_size+validating_set_size)])
  test_targets, test_examples = parse_labels_and_features(signal_dataframe[(training_set_size+validating_set_size):(training_set_size+validating_set_size+test_set_size)])
  nn_classification, accuracy = train_nn_classification_model(
      learning_rate=learning_rate,
      steps=steps,
      batch_size=batch_size,
      hidden_units=model,
      training_examples=training_examples,
      training_targets=training_targets,
      validation_examples=validation_examples,
      validation_targets=validation_targets,
      test_examples=test_examples,
      test_targets=test_targets,
      worksheet = worksheet,
      line = line,
      time_string = time_str)
  return accuracy
  

In [13]:
num_samples = signal_dataframe.shape[0]
learning_rate_steps = [0.1]
data_set_distribution= [[60, 20, 20]]
d = dict()
df = pd.DataFrame(data=d)
# models = [[4096, 2048] , [4096, 2048, 1024]]
models = [[1024, 512] , [1024, 512, 256]]
batch_sizes=[1]
time_string = time.strftime("%Y%m%d-%H%M%S")
name = "test"+time_string+".xlsx"
line=1
shutil.rmtree("saves", ignore_errors=True)
shutil.rmtree("models", ignore_errors=True)
os.makedirs("saves")
with open(name, 'w') as f:
  writer = pd.ExcelWriter(name)
  df.to_excel(writer, sheet_name='Sheet1')
  workbook = writer.book
  worksheet = writer.sheets['Sheet1']
  prepareWorksheet(worksheet)
  for model in models: #number of neurons per layer
    for learning_rate in learning_rate_steps: # learning_rate used for training
      for v in data_set_distribution: # try several dataset_distributions
        for batch_size in batch_sizes:
                last_accuracy=max_accuracy=0
                ckp_best_model=""
                training_set_size = int(num_samples * (v[0]/100))
                validating_set_size = int(num_samples * (v[1]/100))
                test_set_size = int(num_samples * (v[2]/100))
                for x in range(0,3):
                  last_accuracy=train_automated(
                        training_set_size,
                        validating_set_size,
                        test_set_size,
                        learning_rate,
                        training_set_size//batch_size,
                        batch_size,
                        model,
                        worksheet,
                        line = line,
                        time_str=time_string)
                        #insertIntoWorksheet(worksheet, line, "model", learning_rate, training_set_size,validating_set_size , batch_size , 1, "pic1")
                  line = line + 1
                  if max_accuracy < last_accuracy:
                    max_accuracy = last_accuracy
                    ckpname= str(model)+"-"+time_string + "-" + str(x)
                    os.rename("./models/checkpoint", ckpname)
                    if ckp_best_model != "":
                      os.remove(ckp_best_model)
                    ckp_best_model = ckpname
                  else:
                    os.remove("models/checkpoint")   
                line = line + 1 
  writer.save()
  writer.close()
        
      
    

Training model...
LogLoss error (on validation data):
  period 00 : 1.79
  period 01 : 1.18
  period 02 : 0.82
  period 03 : 0.63
  period 04 : 0.46
  period 05 : 0.56
  period 06 : 0.44
  period 07 : 0.36


InvalidArgumentError: ignored